In [ ]:
import numpy as np
import pandas as pd
import json
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
import sklearn
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Import datasets

df = pd.read_csv("/content/drive/MyDrive/MVA/Echantillonage données/HRFlow/dataset/X_train_hYV2vs5.csv", index_col=1)
X_test = pd.read_csv("/content/drive/MyDrive/MVA/Echantillonage données/HRFlow/dataset/X_test_jhAIQq9.csv", index_col=1)
target = pd.read_csv("/content/drive/MyDrive/MVA/Echantillonage données/HRFlow/dataset/y_train_Ga8ie3n.csv", index_col=1)
y_test = pd.read_csv("/content/drive/MyDrive/MVA/Echantillonage données/HRFlow/dataset/y_test_random_5ENWYRq.csv", index_col=1)

In [ ]:
# Clean columns

df.drop(df.columns[0], axis=1, inplace=True)
X_test.drop(X_test.columns[0], axis = 1, inplace=True)
target.drop(target.columns[0], axis=1, inplace=True)
y_test.drop(y_test.columns[0], axis=1, inplace=True)
y_test.head()

,position
id,
29273,Manager
29274,Assistant
29275,Executive
29276,Manager
29277,Manager


In [ ]:
X_test.head()

,employee embedding,company embedding
id,,
29273,"[0.7702940106391907, -0.14788244664669037, 0.4...","[0.5954388976097107, -0.4068148136138916, -0.0..."
29274,"[0.34006059169769287, 0.5506752729415894, -0.2...","[0.5954388976097107, -0.4068148136138916, -0.0..."
29275,"[-0.03870668262243271, 0.13938209414482117, 0....","[0.3746471703052521, 0.14746129512786865, 0.31..."
29276,"[-0.03870668262243271, 0.13938209414482117, 0....","[0.03391633182764053, -0.6105689406394958, 0.2..."
29277,"[0.021639468148350716, -0.39925670623779297, -...","[0.3746471703052521, 0.14746129512786865, 0.31..."


In [ ]:
# Proportion of each class in train set

target.value_counts(normalize=True)

position 
Executive    0.607830
Assistant    0.308236
Director     0.046323
Manager      0.037611
dtype: float64

In [ ]:
# Proportion of each class in test set

y_test.value_counts(normalize=True)

position 
Assistant    0.256858
Manager      0.248123
Director     0.247714
Executive    0.247304
dtype: float64

In [ ]:
# Function to deserialize embeddings
## Update : inutile
def deserialize(embeddings):
  embedding_list = json.loads(embeddings)

  return embedding_list

In [ ]:
df["employee embedding"].dtype

dtype('O')

In [ ]:
# Convert embeddings in numpy array and create one columns per element
employee_embeddings = df['employee embedding'].apply(lambda x: json.loads(x))
employee_embeddings= employee_embeddings.apply(pd.Series)

In [ ]:
employee_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
id,,,,,,,,,,,,,,,,,,,,,
0,0.013263,-0.376167,-0.613827,0.359729,0.193883,0.388050,-0.001953,-0.019567,0.161783,0.073373,...,0.437670,0.081673,0.626443,0.470492,-0.222334,0.611193,-0.885810,-0.196387,-0.415477,-0.889294
1,-0.282738,-0.012908,0.205368,0.482457,0.285612,-0.261424,0.602928,-0.176004,0.351918,-0.483896,...,0.643368,0.117696,-0.855615,0.486371,-0.558203,0.383442,-0.855513,-0.475116,-0.232120,-0.616211
2,0.399990,-0.298473,-0.210973,-0.032957,0.057687,0.421141,0.949937,-0.365188,0.327099,0.015427,...,-0.260633,0.141378,0.834164,0.353678,0.203667,0.677248,-0.837383,-0.713900,0.376695,0.358202
3,0.399990,-0.298473,-0.210973,-0.032957,0.057687,0.421141,0.949937,-0.365188,0.327099,0.015427,...,-0.260633,0.141378,0.834164,0.353678,0.203667,0.677248,-0.837383,-0.713900,0.376695,0.358202
4,0.010124,0.057610,-0.121743,0.057732,0.084081,-0.044404,0.155736,-0.125136,0.237426,-0.125507,...,0.393852,-0.034005,0.310344,0.348569,-0.492618,0.385827,-0.762160,-0.490906,0.328572,-0.649975


In [ ]:
employee_embeddings = employee_embeddings.add_prefix('employee_emb_')

In [ ]:
employee_embeddings.head()

,employee_emb_0,employee_emb_1,employee_emb_2,employee_emb_3,employee_emb_4,employee_emb_5,employee_emb_6,employee_emb_7,employee_emb_8,employee_emb_9,...,employee_emb_22,employee_emb_23,employee_emb_24,employee_emb_25,employee_emb_26,employee_emb_27,employee_emb_28,employee_emb_29,employee_emb_30,employee_emb_31
id,,,,,,,,,,,,,,,,,,,,,
0,0.013263,-0.376167,-0.613827,0.359729,0.193883,0.388050,-0.001953,-0.019567,0.161783,0.073373,...,0.437670,0.081673,0.626443,0.470492,-0.222334,0.611193,-0.885810,-0.196387,-0.415477,-0.889294
1,-0.282738,-0.012908,0.205368,0.482457,0.285612,-0.261424,0.602928,-0.176004,0.351918,-0.483896,...,0.643368,0.117696,-0.855615,0.486371,-0.558203,0.383442,-0.855513,-0.475116,-0.232120,-0.616211
2,0.399990,-0.298473,-0.210973,-0.032957,0.057687,0.421141,0.949937,-0.365188,0.327099,0.015427,...,-0.260633,0.141378,0.834164,0.353678,0.203667,0.677248,-0.837383,-0.713900,0.376695,0.358202
3,0.399990,-0.298473,-0.210973,-0.032957,0.057687,0.421141,0.949937,-0.365188,0.327099,0.015427,...,-0.260633,0.141378,0.834164,0.353678,0.203667,0.677248,-0.837383,-0.713900,0.376695,0.358202
4,0.010124,0.057610,-0.121743,0.057732,0.084081,-0.044404,0.155736,-0.125136,0.237426,-0.125507,...,0.393852,-0.034005,0.310344,0.348569,-0.492618,0.385827,-0.762160,-0.490906,0.328572,-0.649975


In [ ]:
# Convert embeddings in numpy array and create one columns per element
company_embeddings = df['company embedding'].apply(lambda x: json.loads(x))
company_embeddings= company_embeddings.apply(pd.Series)
company_embeddings = company_embeddings.add_prefix('company_emb_')
company_embeddings.head()

,company_emb_0,company_emb_1,company_emb_2,company_emb_3,company_emb_4,company_emb_5,company_emb_6,company_emb_7,company_emb_8,company_emb_9,...,company_emb_22,company_emb_23,company_emb_24,company_emb_25,company_emb_26,company_emb_27,company_emb_28,company_emb_29,company_emb_30,company_emb_31
id,,,,,,,,,,,,,,,,,,,,,
0,0.397617,0.011816,0.187563,0.280758,-0.343823,-0.260132,-0.243069,-0.050393,0.159177,-0.131660,...,0.496415,-0.295326,0.294123,0.566333,-0.450409,0.667816,-0.907596,-0.421868,-0.320554,-0.404289
1,0.310513,-0.338024,0.023329,0.698560,0.551937,0.773255,0.141423,-0.082342,0.051717,0.063137,...,0.534470,-0.209862,0.664622,0.539178,-0.367445,0.639607,-0.869631,-0.154816,0.075938,-0.639021
2,0.197047,0.062400,0.256581,0.255206,-0.300442,0.131297,0.729090,-0.043291,0.219654,-0.204823,...,0.125662,0.140065,0.080729,0.233294,-0.238041,0.564176,-0.930845,-0.559797,0.103106,-0.149208
3,0.650161,-0.177572,0.144814,0.248895,0.105174,-0.358299,-0.119391,-0.165776,0.021271,-0.155839,...,0.413645,-0.109720,0.322721,0.508323,-0.369104,0.496631,-0.881074,-0.056100,0.556903,-0.202640
4,0.596681,-0.061621,0.056970,0.388950,-0.100503,-0.069174,0.446821,0.173248,0.205579,-0.176114,...,0.416849,0.408343,0.337121,0.392495,-0.265649,0.664976,-0.922024,-0.505522,-0.364163,-0.073651


In [ ]:
result_df = pd.concat([employee_embeddings, company_embeddings], axis=1, join='inner')

In [ ]:
result_df.head()

,employee_emb_0,employee_emb_1,employee_emb_2,employee_emb_3,employee_emb_4,employee_emb_5,employee_emb_6,employee_emb_7,employee_emb_8,employee_emb_9,...,company_emb_22,company_emb_23,company_emb_24,company_emb_25,company_emb_26,company_emb_27,company_emb_28,company_emb_29,company_emb_30,company_emb_31
id,,,,,,,,,,,,,,,,,,,,,
0,0.013263,-0.376167,-0.613827,0.359729,0.193883,0.388050,-0.001953,-0.019567,0.161783,0.073373,...,0.496415,-0.295326,0.294123,0.566333,-0.450409,0.667816,-0.907596,-0.421868,-0.320554,-0.404289
1,-0.282738,-0.012908,0.205368,0.482457,0.285612,-0.261424,0.602928,-0.176004,0.351918,-0.483896,...,0.534470,-0.209862,0.664622,0.539178,-0.367445,0.639607,-0.869631,-0.154816,0.075938,-0.639021
2,0.399990,-0.298473,-0.210973,-0.032957,0.057687,0.421141,0.949937,-0.365188,0.327099,0.015427,...,0.125662,0.140065,0.080729,0.233294,-0.238041,0.564176,-0.930845,-0.559797,0.103106,-0.149208
3,0.399990,-0.298473,-0.210973,-0.032957,0.057687,0.421141,0.949937,-0.365188,0.327099,0.015427,...,0.413645,-0.109720,0.322721,0.508323,-0.369104,0.496631,-0.881074,-0.056100,0.556903,-0.202640
4,0.010124,0.057610,-0.121743,0.057732,0.084081,-0.044404,0.155736,-0.125136,0.237426,-0.125507,...,0.416849,0.408343,0.337121,0.392495,-0.265649,0.664976,-0.922024,-0.505522,-0.364163,-0.073651


In [ ]:
# Same for test set
company_embeddings_test = X_test['company embedding'].apply(lambda x: json.loads(x))
company_embeddings_test= company_embeddings_test.apply(pd.Series)
company_embeddings_test = company_embeddings_test.add_prefix('company_emb_')

employee_embeddings_test = X_test['employee embedding'].apply(lambda x: json.loads(x))
employee_embeddings_test = employee_embeddings_test.apply(pd.Series)
employee_embeddings_test = employee_embeddings_test.add_prefix('employee_emb_')

In [ ]:
company_embeddings_test.head()

,company_emb_0,company_emb_1,company_emb_2,company_emb_3,company_emb_4,company_emb_5,company_emb_6,company_emb_7,company_emb_8,company_emb_9,...,company_emb_22,company_emb_23,company_emb_24,company_emb_25,company_emb_26,company_emb_27,company_emb_28,company_emb_29,company_emb_30,company_emb_31
id,,,,,,,,,,,,,,,,,,,,,
29273,0.595439,-0.406815,-0.020347,0.398171,-0.040953,-0.427655,-0.210465,-0.060725,0.343200,-0.137855,...,0.521330,0.350061,0.162991,0.504313,-0.371429,0.675732,-0.967248,-0.196321,0.080971,-0.336691
29274,0.595439,-0.406815,-0.020347,0.398171,-0.040953,-0.427655,-0.210465,-0.060725,0.343200,-0.137855,...,0.521330,0.350061,0.162991,0.504313,-0.371429,0.675732,-0.967248,-0.196321,0.080971,-0.336691
29275,0.374647,0.147461,0.310082,0.475489,-0.376326,0.541714,0.881498,0.004099,0.157505,0.215978,...,0.441937,0.111995,0.694086,0.436403,-0.137214,0.280484,-0.921925,-0.216101,-0.433705,0.196600
29276,0.033916,-0.610569,0.208433,0.299742,0.034581,0.600358,-0.443489,0.028966,0.178381,-0.225869,...,0.312529,0.038193,-0.368810,0.342731,-0.081778,0.667781,-0.930811,-0.222494,-0.186185,-0.379813
29277,0.374647,0.147461,0.310082,0.475489,-0.376326,0.541714,0.881498,0.004099,0.157505,0.215978,...,0.441937,0.111995,0.694086,0.436403,-0.137214,0.280484,-0.921925,-0.216101,-0.433705,0.196600


In [ ]:
employee_embeddings_test.head()

,employee_emb_0,employee_emb_1,employee_emb_2,employee_emb_3,employee_emb_4,employee_emb_5,employee_emb_6,employee_emb_7,employee_emb_8,employee_emb_9,...,employee_emb_22,employee_emb_23,employee_emb_24,employee_emb_25,employee_emb_26,employee_emb_27,employee_emb_28,employee_emb_29,employee_emb_30,employee_emb_31
id,,,,,,,,,,,,,,,,,,,,,
29273,0.770294,-0.147882,0.486307,0.389991,-0.450313,-0.526806,0.998577,0.304267,0.073247,-0.047410,...,-0.122459,-0.153623,0.224232,0.613452,0.294292,0.362747,-0.941970,-0.618015,-0.861649,0.941208
29274,0.340061,0.550675,-0.273903,0.265983,-0.219236,0.081678,0.179793,-0.025414,0.186707,-0.193626,...,0.435066,-0.396087,0.409665,0.411913,-0.560096,0.708977,-0.808181,-0.676881,-0.202634,-0.679211
29275,-0.038707,0.139382,0.114205,-0.258506,0.239997,0.411867,0.133353,-0.058503,0.151351,-0.164319,...,0.479362,0.320472,-0.072223,0.446141,0.206721,0.680315,-0.952567,-0.144111,0.110263,-0.931206
29276,-0.038707,0.139382,0.114205,-0.258506,0.239997,0.411867,0.133353,-0.058503,0.151351,-0.164319,...,0.479362,0.320472,-0.072223,0.446141,0.206721,0.680315,-0.952567,-0.144111,0.110263,-0.931206
29277,0.021639,-0.399257,-0.742458,0.633686,0.082964,0.399026,0.496338,-0.032831,0.067287,0.115868,...,0.502544,0.294021,0.300221,0.754463,0.117947,0.788333,-0.931425,-0.611128,-0.600412,-0.812737


In [ ]:
# Concatenate employee and company embeddings

result_df_test = pd.concat([employee_embeddings_test, company_embeddings_test], axis=1, join='inner')

In [ ]:
result_df_test.head()

,employee_emb_0,employee_emb_1,employee_emb_2,employee_emb_3,employee_emb_4,employee_emb_5,employee_emb_6,employee_emb_7,employee_emb_8,employee_emb_9,...,company_emb_22,company_emb_23,company_emb_24,company_emb_25,company_emb_26,company_emb_27,company_emb_28,company_emb_29,company_emb_30,company_emb_31
id,,,,,,,,,,,,,,,,,,,,,
29273,0.770294,-0.147882,0.486307,0.389991,-0.450313,-0.526806,0.998577,0.304267,0.073247,-0.047410,...,0.521330,0.350061,0.162991,0.504313,-0.371429,0.675732,-0.967248,-0.196321,0.080971,-0.336691
29274,0.340061,0.550675,-0.273903,0.265983,-0.219236,0.081678,0.179793,-0.025414,0.186707,-0.193626,...,0.521330,0.350061,0.162991,0.504313,-0.371429,0.675732,-0.967248,-0.196321,0.080971,-0.336691
29275,-0.038707,0.139382,0.114205,-0.258506,0.239997,0.411867,0.133353,-0.058503,0.151351,-0.164319,...,0.441937,0.111995,0.694086,0.436403,-0.137214,0.280484,-0.921925,-0.216101,-0.433705,0.196600
29276,-0.038707,0.139382,0.114205,-0.258506,0.239997,0.411867,0.133353,-0.058503,0.151351,-0.164319,...,0.312529,0.038193,-0.368810,0.342731,-0.081778,0.667781,-0.930811,-0.222494,-0.186185,-0.379813
29277,0.021639,-0.399257,-0.742458,0.633686,0.082964,0.399026,0.496338,-0.032831,0.067287,0.115868,...,0.441937,0.111995,0.694086,0.436403,-0.137214,0.280484,-0.921925,-0.216101,-0.433705,0.196600


In [ ]:
#Encode the labels

label_mapping = {'Assistant': 0, 'Executive': 1, 'Manager': 2, 'Director': 3}

target_encoded = [label_mapping[label] for label in target['position']]
target_encoded = pd.DataFrame({'position':target_encoded}, index = range(29273))
target_encoded.index.name = 'id'

y_test_encoded = [label_mapping[label] for label in y_test['position']]
y_test_encoded = pd.DataFrame({'position':y_test_encoded}, index = range(29273, 36600))
y_test_encoded.index.name = 'id'


In [ ]:
# Create dataframe result_df2 with variables as list
df2 = pd.DataFrame()
df2['company_embedding'] = df['company embedding'].apply(lambda x: json.loads(x))
df2['employee_embedding'] = df['employee embedding'].apply(lambda x: json.loads(x))

In [ ]:
# See if there are several times the same employee
duplicate_rows = df2[df2.duplicated(subset=['employee_embedding'], keep=False)]

# Affichez les lignes dupliquées
print("Dublicated lines :")
print(duplicate_rows)
merged_df = pd.merge(duplicate_rows, target, on='id')

Lignes dupliquées :
                                       company_embedding  \
id                                                         
2      [0.1970466524362564, 0.062399972230196, 0.2565...   
3      [0.6501612067222595, -0.17757245898246765, 0.1...   
8      [0.6174620389938354, -0.5719079375267029, 0.16...   
9      [-0.2682649791240692, 0.3388311266899109, 0.77...   
10     [0.8214082717895508, -0.7016443014144897, 0.51...   
...                                                  ...   
29267  [0.46065452694892883, -0.009176196530461311, 0...   
29268  [0.8371062278747559, 0.16954681277275085, -0.0...   
29269  [0.2601408362388611, -0.29598891735076904, -0....   
29270  [0.03531331196427345, -0.6426340341567993, -0....   
29271  [0.3410402536392212, 0.22074635326862335, 0.09...   

                                      employee_embedding  
id                                                        
2      [0.3999897241592407, -0.2984728515148163, -0.2...  
3      [0.399989724159

In [ ]:
merged_df.head(30)

,company_embedding,employee_embedding,position
id,,,
2,"[0.1970466524362564, 0.062399972230196, 0.2565...","[0.3999897241592407, -0.2984728515148163, -0.2...",Executive
3,"[0.6501612067222595, -0.17757245898246765, 0.1...","[0.3999897241592407, -0.2984728515148163, -0.2...",Executive
8,"[0.6174620389938354, -0.5719079375267029, 0.16...","[-0.06439319252967834, 0.03868088498711586, 0....",Assistant
9,"[-0.2682649791240692, 0.3388311266899109, 0.77...","[-0.06439319252967834, 0.03868088498711586, 0....",Assistant
10,"[0.8214082717895508, -0.7016443014144897, 0.51...","[-0.06439319252967834, 0.03868088498711586, 0....",Executive
11,"[0.633974015712738, -0.6420604586601257, 0.312...","[-0.06439319252967834, 0.03868088498711586, 0....",Executive
12,"[-0.09147301316261292, -0.1544800102710724, -0...","[-0.06439319252967834, 0.03868088498711586, 0....",Executive
13,"[0.5169798135757446, -0.531933605670929, -0.29...","[-0.06439319252967834, 0.03868088498711586, 0....",Executive
14,"[-0.0650421679019928, -0.29614022374153137, 0....","[-0.06439319252967834, 0.03868088498711586, 0....",Assistant


# Validation set

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(result_df, target, test_size=0.3)

In [ ]:
print(X_train.shape)

(20491, 64)


In [ ]:
y_train.head(10)

,position
id,
19694,Executive
5353,Executive
16655,Assistant
28929,Executive
14775,Executive
19302,Manager
14258,Executive
25648,Executive
20797,Executive


In [ ]:
target

,position
id,
0,Executive
1,Executive
2,Executive
3,Executive
4,Executive
...,...
29268,Assistant
29269,Executive
29270,Executive


# XGBOOST

In [ ]:
xgb_classifier = XGBClassifier(max_depth=7, learning_rate=0.01, n_estimators = 200)
xgb_classifier.fit(result_df, target_encoded)
y_pred = xgb_classifier.predict(result_df_test)

In [ ]:
xgb_classifier = XGBClassifier()

# Grid of parameters to optimize
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [50, 100, 200]
}

# Grdi Search Cv
grid_search = GridSearchCV(estimator=xgb_classifier, param_grid=param_grid, cv=3, scoring='f1_macro')

# Effectuer la recherche par grille sur les données d'entraînement
grid_search.fit(result_df, target_encoded)

# Afficher les meilleurs paramètres
best_params = grid_search.best_params_
print("Meilleurs paramètres:", best_params)

# Utiliser le modèle optimisé pour les prédictions sur les données de test
y_pred = grid_search.predict(result_df_test)

KeyboardInterrupt: 

In [ ]:
print(f1_score(y_test_encoded, y_pred, average="macro"))

0.12093165590819216


In [ ]:
result_df2 = df.copy()

In [ ]:
result_df2['company embedding']


id
0        [0.39761704206466675, 0.011816106736660004, 0....
1        [0.31051310896873474, -0.33802372217178345, 0....
2        [0.1970466524362564, 0.062399972230196, 0.2565...
3        [0.6501612067222595, -0.17757245898246765, 0.1...
4        [0.5966811180114746, -0.06162050738930702, 0.0...
                               ...                        
29268    [0.8371062278747559, 0.16954681277275085, -0.0...
29269    [0.2601408362388611, -0.29598891735076904, -0....
29270    [0.03531331196427345, -0.6426340341567993, -0....
29271    [0.3410402536392212, 0.22074635326862335, 0.09...
29272    [0.14475694298744202, -0.45146191120147705, -0...
Name: company embedding, Length: 29273, dtype: object

In [ ]:
result_df2.dtypes

company_embedding     object
employee_embedding    object
dtype: object

In [ ]:
result_df2['company_embedding'] = df['company embedding'].apply(deserialize)

In [ ]:
result_df2['employee_embedding'] = df['employee embedding'].apply(deserialize)

In [ ]:
result_df2.drop(['company embedding', 'employee embedding'], axis=1,inplace=True)

In [ ]:
df2 = pd.DataFrame()

In [ ]:
df2['employee_embedding'] = df['employee embedding'].apply(lambda x: json.loads(x))

In [ ]:
df2.dtypes

employee_embedding    object
dtype: object

In [ ]:
df2

,employee_embedding
id,
0,"[0.0132625512778759, -0.37616726756095886, -0...."
1,"[-0.2827381491661072, -0.012908441945910454, 0..."
2,"[0.3999897241592407, -0.2984728515148163, -0.2..."
3,"[0.3999897241592407, -0.2984728515148163, -0.2..."
4,"[0.010124210268259048, 0.05761045217514038, -0..."
...,...
29268,"[-0.2505040168762207, -0.20676210522651672, 0...."
29269,"[-0.2505040168762207, -0.20676210522651672, 0...."
29270,"[-0.2505040168762207, -0.20676210522651672, 0...."


In [ ]:
# Utilisez duplicated pour identifier les lignes avec les mêmes embeddings pour le même employé
duplicate_rows = result_df2[result_df2.duplicated(subset=['employee_embedding'], keep=False)]

# Affichez les lignes dupliquées
print("Lignes dupliquées :")
print(duplicate_rows)
merged_df = pd.merge(duplicate_rows, target, on='id')

Lignes dupliquées :
                                       company_embedding  \
id                                                         
2      [0.1970466524362564, 0.062399972230196, 0.2565...   
3      [0.6501612067222595, -0.17757245898246765, 0.1...   
8      [0.6174620389938354, -0.5719079375267029, 0.16...   
9      [-0.2682649791240692, 0.3388311266899109, 0.77...   
10     [0.8214082717895508, -0.7016443014144897, 0.51...   
...                                                  ...   
29267  [0.46065452694892883, -0.009176196530461311, 0...   
29268  [0.8371062278747559, 0.16954681277275085, -0.0...   
29269  [0.2601408362388611, -0.29598891735076904, -0....   
29270  [0.03531331196427345, -0.6426340341567993, -0....   
29271  [0.3410402536392212, 0.22074635326862335, 0.09...   

                                      employee_embedding  
id                                                        
2      [0.3999897241592407, -0.2984728515148163, -0.2...  
3      [0.399989724159

In [ ]:
merged_df.head(30)

,company_embedding,employee_embedding,position
id,,,
2,"[0.1970466524362564, 0.062399972230196, 0.2565...","[0.3999897241592407, -0.2984728515148163, -0.2...",Executive
3,"[0.6501612067222595, -0.17757245898246765, 0.1...","[0.3999897241592407, -0.2984728515148163, -0.2...",Executive
8,"[0.6174620389938354, -0.5719079375267029, 0.16...","[-0.06439319252967834, 0.03868088498711586, 0....",Assistant
9,"[-0.2682649791240692, 0.3388311266899109, 0.77...","[-0.06439319252967834, 0.03868088498711586, 0....",Assistant
10,"[0.8214082717895508, -0.7016443014144897, 0.51...","[-0.06439319252967834, 0.03868088498711586, 0....",Executive
11,"[0.633974015712738, -0.6420604586601257, 0.312...","[-0.06439319252967834, 0.03868088498711586, 0....",Executive
12,"[-0.09147301316261292, -0.1544800102710724, -0...","[-0.06439319252967834, 0.03868088498711586, 0....",Executive
13,"[0.5169798135757446, -0.531933605670929, -0.29...","[-0.06439319252967834, 0.03868088498711586, 0....",Executive
14,"[-0.0650421679019928, -0.29614022374153137, 0....","[-0.06439319252967834, 0.03868088498711586, 0....",Assistant
